In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
from src.problem import msQuadratic
model = msQuadratic()

In [5]:
# parameters as input data
p_low, p_high = 0.0, 1.0
p_train = np.random.uniform(p_low, p_high, (train_size, 2)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 2)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 2)).astype(np.float32)

In [6]:
from pyomo import opt as po
# feasibility filter
def feasibility_filter(p_data):
    for i, p in tqdm(enumerate(p_data)):
        # set param
        model.set_param_val({"p":p})
        # solve
        _, _ = model.solve("scip")
        while model.res.solver.termination_condition == po.TerminationCondition.infeasible:
            # generate a new p
            p = np.random.uniform(p_low, p_high, 2).astype(np.float32)
            # set param
            model.set_param_val({"p":p})
            # solve
            _, _ = model.solve("scip")
        # update p
        p_data[i] = p

# make all data feasible
feasibility_filter(p_train)
feasibility_filter(p_test)
feasibility_filter(p_dev)

0it [00:00, ?it/s]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


3000it [05:51,  8.53it/s]
1000it [01:44,  9.60it/s]
1000it [01:46,  9.43it/s]


In [7]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train}, name="train")
data_test = DictDataset({"p":p_test}, name="test")
data_dev = DictDataset({"p":p_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p in tqdm(p_test):
    model.set_param_val({"p":p})
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/qp_exact.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:24<00:00, 11.85it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     -0.300508               0.0      0.083594
std       0.017031               0.0      0.037765
min      -0.310000               0.0      0.058508
25%      -0.310000               0.0      0.061619
50%      -0.310000               0.0      0.073384
75%      -0.300000               0.0      0.077798
max      -0.210132               0.0      0.244359
Number of infeasible solution: 0


## Heuristic

In [9]:
model_heur = model.first_solution_heuristic()

In [10]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p in tqdm(p_test):
    model_heur.set_param_val({"p":p})
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/qp_heur.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:19<00:00, 12.56it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     -0.235842               0.0      0.078855
std       0.117741               0.0      0.034860
min      -0.310000               0.0      0.058023
25%      -0.300000               0.0      0.061376
50%      -0.300000               0.0      0.062733
75%      -0.260710               0.0      0.076504
max       0.000000               0.0      0.186517
Number of infeasible solution: 0


## Learnable Rounding

In [11]:
# hyperparameters
penalty_weight = 10   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 3       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-2             # learning rate
batch_size = 64       # batch size

In [12]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# define quadratic objective functions and constraints for both problem types
obj_rel, constrs_rel = nmQuadratic(["x"], ["p"], penalty_weight=penalty_weight)
obj_rnd, constrs_rnd = nmQuadratic(["x_rnd"], ["p"], penalty_weight=penalty_weight)
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=2, outsize=4, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=6, hidden_dims=[hsize]*hlayers_rnd, output_dim=4)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p"], var_keys=["x"], output_keys=["x_rnd"],
                       int_ind={"x":[2,3]}, continuous_update=False, name="round")
# build neuromancer problem for rounding
components = [smap, rnd]
loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
problem = nm.problem.Problem(components, loss)

In [13]:
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(problem.parameters(), lr=lr)
# create a trainer for the problem
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test, optimizer, 
                            epochs=epochs, patience=patience, warmup=warmup)
# training for the rounding problem
best_model = trainer.train()
# load best model dict
problem.load_state_dict(best_model)

epoch: 0  train_loss: 1.9006130695343018
epoch: 1  train_loss: 0.8498107194900513
epoch: 2  train_loss: 0.8236396312713623
epoch: 3  train_loss: 0.3078206181526184
epoch: 4  train_loss: 0.360994815826416
epoch: 5  train_loss: 0.6205000877380371
epoch: 6  train_loss: 0.6599565148353577
epoch: 7  train_loss: 1.07606840133667
epoch: 8  train_loss: 0.16626447439193726
epoch: 9  train_loss: 0.037488069385290146
epoch: 10  train_loss: 0.011245842091739178
epoch: 11  train_loss: 0.1659967303276062
epoch: 12  train_loss: 0.13184885680675507
epoch: 13  train_loss: 0.06834141910076141
epoch: 14  train_loss: 0.08455299586057663
epoch: 15  train_loss: 0.07996489852666855
epoch: 16  train_loss: 0.01878316141664982
epoch: 17  train_loss: 0.18229149281978607
epoch: 18  train_loss: 0.19630023837089539
epoch: 19  train_loss: 0.011691409163177013
epoch: 20  train_loss: 0.13464005291461945
epoch: 21  train_loss: 0.16005240380764008
epoch: 22  train_loss: 0.10071253031492233
epoch: 23  train_loss: 0.24319

<All keys matched successfully>

In [14]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p in tqdm(p_test):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    # infer
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    # assign params
    model.set_param_val({"p":p})
    # assign vars
    x = output["test_x_rnd"]
    for i in range(4):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/qp_nm.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 78.16it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.289489          0.000051      0.011645
std       0.019035          0.001018      0.003067
min      -0.297858          0.000000      0.006435
25%      -0.297858          0.000000      0.009306
50%      -0.297858          0.000000      0.010991
75%      -0.291715          0.000000      0.013458
max      -0.186733          0.029536      0.031913
Number of infeasible solution: 4
